In [1]:
import requests
import json
import docx2txt
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#url to job search API
url = "https://jsearch.p.rapidapi.com/search"

#time = "3days"
time = "week"

#query criteria 
querystring = {"query":"data analyst data scientist","page":"1","num_pages":"9","date_posted":time,
               "remote_jobs_only":"true","employment_types":"FULLTIME"}

#keys
headers = {
    "X-RapidAPI-Key": "XXXXXXX",
    "X-RapidAPI-Host": "jsearch.p.rapidapi.com"}

response = requests.request("GET", url, headers=headers, params=querystring).json()


In [4]:
#resuem to check against job descriptions
resume = docx2txt.process("King_Resume_ds_2_final.docx")

#dataframe with headers
job_detail = pd.DataFrame(columns=['Date','Company','Position','Score','Apply','Link', 'Job ID'])

#loop through entir list of company job postions
for i in range (len(response['data'])):
   
    #list to temp hold qualifications and responsibilities
    desc = []
    
    #job info to move to dataframe
    date_time = response['data'][i]['job_posted_at_datetime_utc']
    date = date_time[:10]
    name = response['data'][i]['employer_name']
    title = response['data'][i]['job_title']
    link = response['data'][i]['job_apply_link']
    Jobid = response['data'][i]['job_id']
  
    #not all job postins have qualifications and responsibilities
    #will try to pull these first if not will pull entire job description
    #job descriptions have exrta word that could effect ATS score that is 
    #way it is not prefered
    try:
        
        Q=response['data'][i]['job_highlights']['Qualifications']
        R=response['data'][i]['job_highlights']['Responsibilities']
        #combines the to into one list 
        desc = Q + R   
    
    except:
        desc = (response['data'][i]['job_description'])
    
    #creates a temp text.txt file to store job description
    f = open('text.txt','w')
    
    #writes job description to file element by element
    for i in desc:
        #if there is a element that can not be read it skips it and goes to the next one
        try:
            f.write(i)
        except:
            continue
    #closes the writing of the file       
    f.close()
    
    #opens the new text file with the job description
    jobDec=open('text.txt').read()
  
    #starts comparison of resume to job description using text vectorization
    text =[resume,jobDec]
    cv = CountVectorizer(analyzer='word',stop_words= 'english')
    cvMatrix = cv.fit_transform(text)

    match = cosine_similarity(cvMatrix)
    
    #ats score of resume to job description.  Higher the score, the more similar they are 
    ats = match[0][1]
    
    #deleted text.txt file 
    os.remove('text.txt')
    
    #comple row entery in to dataframe
    row = {'Date':date,'Company': name, 'Link': link, 'Position': title,'Score':ats, 'Job ID':Jobid}
    
    #adds new row into dataframe
    job_detail = job_detail.append(row, ignore_index = True)
   

In [5]:
#checks to see if the job Search file exists, if not creates the first version

if os.path.isfile("Job Search.xlsx") == True:
    #opens job serach file
    exFile = pd.read_excel("Job Search.xlsx", index_col=0) 
    
    #loop through the all the jobs and see find ones that are
    #not on the list
    for i in job_detail.index:
        newDate = job_detail['Date'][i]
        newCompany = job_detail['Company'][i]
        newLink = job_detail['Link'][i]
        newPosition = job_detail['Position'][i]
        newSocre = job_detail['Score'][i]
        newJobid = job_detail['Job ID'][i]
    
        #if it finds a match it is not added
        if newJobid in exFile['Job ID'].values:
            continue
            
        else:
            #add new jod to list
            addrow = {'Date':newDate,'Company': newCompany, 'Link': newLink, 
                      'Position': newPosition,'Score':newSocre, 'Job ID':newJobid}
            exFile = exFile.append(addrow, ignore_index = True)


    #sorts the new complete list
    exFile=exFile.sort_values(by='Score', ascending=False)
    #removes old file
    os.remove('Job Search.xlsx')
    #creates new excel file 
    exFile.to_excel("Job Search.xlsx",freeze_panes=(1,0))  
        
else:
   
    #sorts data frame by highest ats score
    job_detail=job_detail.sort_values(by='Score', ascending=False)
    #Creates excel file of job descriptions
    job_detail.to_excel("Job Search.xlsx",freeze_panes=(1,0))

print("Job file created")

Job file created
